In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [2]:
final = pd.read_csv('MME_final.csv', encoding = 'euc-kr', low_memory = False)
final

,기수,NIHID,SEX,VISITALL,PHYSTB,PHYSIT,PHYACTL,PHYACTM,PHYACTH,AEROBFQ,...,FMCDMAG,FMHEA,FMFHEA,FMFHEAAG,FMMHEA,FMMHEAAG,FMBHEA,FMBHEAAG,FMCHEA,FMCHEAAG
0,A01,NIH1604000095,2,6,1,5,7,6,4,99999,...,66666,1,1,77777,1,77777,1,77777,66666,66666
1,A01,NIH1604000171,1,7,0,3,8,8,0,99999,...,66666,1,1,77777,1,77777,1,77777,66666,66666
2,A01,NIH1604000338,1,7,2,5,3,0,7,99999,...,66666,1,1,77777,1,77777,1,77777,66666,66666
3,A01,NIH1604000362,2,7,0,5,4,2,2,99999,...,66666,1,1,77777,1,77777,1,77777,66666,66666
4,A01,NIH1604000424,1,4,8,5,8,8,0,99999,...,66666,1,1,77777,1,77777,1,77777,66666,66666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,NIH1604999643,2,7,66666,66666,66666,66666,66666,66666,...,77777,1,1,77777,1,77777,1,77777,1,77777
70206,A07,NIH1604999732,1,6,66666,66666,66666,66666,66666,66666,...,77777,1,1,77777,1,77777,1,77777,1,77777
70207,A07,NIH1604999772,1,6,66666,66666,66666,66666,66666,66666,...,77777,1,1,77777,1,77777,1,77777,1,77777
70208,A07,NIH1604999929,2,4,66666,66666,66666,66666,66666,66666,...,77777,1,1,77777,1,77777,1,77777,1,77777


In [3]:
col = ['기수', 'EDATE', # 10.19 EDATE 추가
       'NIHID', 'AGE', 'SEX', 'CREATININ_TR', 'CREATININ_ORI',
       'SBP_L', 'SBP_R', 'DBP_L', 'DBP_R', 'HEIGHT', 'WEIGHT', 
'WAIST',
'GLU0_ORI',
'R_GTP_TR',   # 10.19 GTP TR 추가, GTP ORI 삭제
'AST_ORI',
'ALT_ORI',
'TCHL_ORI',
'HDL_ORI',

'TRIGLY_ORI',
'HB_ORI',

'SMOKE',
'DRUGINS',
'DRUGHT',
'TREATD5',
'DRUGICD',
'DRUGLP',
       
'FMHTN',
'FMHEA',
'FMDM',
'PRT16_U',

'TREATD14',
'KID',
'KIDAG',
'KIDCU', 
       
'TOTALC', 'MET', 'PHYACTL', 'PHYACTM', 'PHYACTH', 
       
'BODYFAT'    # 10.19 BODYFAT 추가
]

In [4]:
df = final[col]
df

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,SBP_L,SBP_R,DBP_L,...,TREATD14,KID,KIDAG,KIDCU,TOTALC,MET,PHYACTL,PHYACTM,PHYACTH,BODYFAT
0,A01,200209,NIH1604000095,43,2,0.90,0.90,98,94,66,...,1,1,77777,66666,1.30,19740.0,7,6,4,13.4
1,A01,200201,NIH1604000171,42,1,1.00,1.00,110,80,80,...,1,1,77777,66666,42.68,17587.5,8,8,0,15.7
2,A01,200210,NIH1604000338,69,1,0.90,0.90,138,138,70,...,99999,1,77777,66666,0.00,16380.0,3,0,7,99999.0
3,A01,200201,NIH1604000362,48,2,0.70,0.60,104,100,76,...,99999,1,77777,66666,0.00,7560.0,4,2,2,14.7
4,A01,200205,NIH1604000424,47,1,0.70,0.60,132,130,84,...,99999,1,77777,66666,21.22,18375.0,8,8,0,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303,NIH1604999643,55,2,0.88,0.86,90,94,60,...,1,1,77777,77777,0.00,66666.0,66666,66666,66666,17.9
70206,A07,201407,NIH1604999732,75,1,1.06,1.02,134,128,66,...,1,1,77777,77777,18.81,66666.0,66666,66666,66666,10.5
70207,A07,201409,NIH1604999772,55,1,1.54,1.45,99999,150,99999,...,1,1,77777,77777,8.68,66666.0,66666,66666,66666,13.9
70208,A07,201403,NIH1604999929,53,2,0.89,0.87,110,114,70,...,1,1,77777,77777,3.47,66666.0,66666,66666,66666,27.1


In [5]:
# 55555 ~ 99999 전부 NaN으로 대체
df = df[df[col] != 55555]

df = df[df[col] != 66666]

df = df[df[col] != 77777]

df = df[df[col] != 88888]

df = df[df[col] != 99999]

In [6]:
df

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,SBP_L,SBP_R,DBP_L,...,TREATD14,KID,KIDAG,KIDCU,TOTALC,MET,PHYACTL,PHYACTM,PHYACTH,BODYFAT
0,A01,200209.0,NIH1604000095,43.0,2,0.90,0.90,98.0,94.0,66.0,...,1.0,1.0,NaN,NaN,1.30,19740.0,7.0,6.0,4.0,13.4
1,A01,200201.0,NIH1604000171,42.0,1,1.00,1.00,110.0,80.0,80.0,...,1.0,1.0,NaN,NaN,42.68,17587.5,8.0,8.0,0.0,15.7
2,A01,200210.0,NIH1604000338,69.0,1,0.90,0.90,138.0,138.0,70.0,...,NaN,1.0,NaN,NaN,0.00,16380.0,3.0,0.0,7.0,NaN
3,A01,200201.0,NIH1604000362,48.0,2,0.70,0.60,104.0,100.0,76.0,...,NaN,1.0,NaN,NaN,0.00,7560.0,4.0,2.0,2.0,14.7
4,A01,200205.0,NIH1604000424,47.0,1,0.70,0.60,132.0,130.0,84.0,...,NaN,1.0,NaN,NaN,21.22,18375.0,8.0,8.0,0.0,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,0.88,0.86,90.0,94.0,60.0,...,1.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,17.9
70206,A07,201407.0,NIH1604999732,75.0,1,1.06,1.02,134.0,128.0,66.0,...,1.0,1.0,NaN,NaN,18.81,NaN,NaN,NaN,NaN,10.5
70207,A07,201409.0,NIH1604999772,55.0,1,1.54,1.45,NaN,150.0,NaN,...,1.0,1.0,NaN,NaN,8.68,NaN,NaN,NaN,NaN,13.9
70208,A07,201403.0,NIH1604999929,53.0,2,0.89,0.87,110.0,114.0,70.0,...,1.0,1.0,NaN,NaN,3.47,NaN,NaN,NaN,NaN,27.1


In [7]:
# check = df[['기수', 'EDATE', 'R_GTP_ORI', 'R_GTP_TR']]
# check.groupby(['기수']).count()

# check.loc[check['기수'] == 'A01']

# 8 * 1.004 - 2.98

In [8]:
# 신체활동 categorical -> 시간 (분) -> 시간 (시)로 바꾸는 함수

def calculate_physical_activity(df, colname): 
    for idx in range(len(df)): 
        if df[colname][idx] == 0: 
            df[colname][idx] = 0
        if df[colname][idx] == 1: 
            df[colname][idx] = 15 / 60
        if df[colname][idx] == 2: 
            df[colname][idx] = 45 / 60
        if df[colname][idx] == 3: 
            df[colname][idx] = 75 / 60
        if df[colname][idx] == 4: 
            df[colname][idx] = 105 / 60
        if df[colname][idx] == 5: 
            df[colname][idx] = 150 / 60
        if df[colname][idx] == 6: 
            df[colname][idx] = 210 / 60
        if df[colname][idx] == 7: 
            df[colname][idx] = 270 / 60
        if df[colname][idx] == 8: 
            df[colname][idx] = 330 / 60
    return df

In [9]:
df = calculate_physical_activity(df, 'PHYACTL')
df

<ipython-input-8-cc33c9591cc8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 270 / 60
<ipython-input-8-cc33c9591cc8>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 330 / 60
<ipython-input-8-cc33c9591cc8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 75 / 60
<ipython-input-8-cc33c9591cc8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,SBP_L,SBP_R,DBP_L,...,TREATD14,KID,KIDAG,KIDCU,TOTALC,MET,PHYACTL,PHYACTM,PHYACTH,BODYFAT
0,A01,200209.0,NIH1604000095,43.0,2,0.90,0.90,98.0,94.0,66.0,...,1.0,1.0,NaN,NaN,1.30,19740.0,4.50,6.0,4.0,13.4
1,A01,200201.0,NIH1604000171,42.0,1,1.00,1.00,110.0,80.0,80.0,...,1.0,1.0,NaN,NaN,42.68,17587.5,5.50,8.0,0.0,15.7
2,A01,200210.0,NIH1604000338,69.0,1,0.90,0.90,138.0,138.0,70.0,...,NaN,1.0,NaN,NaN,0.00,16380.0,1.25,0.0,7.0,NaN
3,A01,200201.0,NIH1604000362,48.0,2,0.70,0.60,104.0,100.0,76.0,...,NaN,1.0,NaN,NaN,0.00,7560.0,1.75,2.0,2.0,14.7
4,A01,200205.0,NIH1604000424,47.0,1,0.70,0.60,132.0,130.0,84.0,...,NaN,1.0,NaN,NaN,21.22,18375.0,5.50,8.0,0.0,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,0.88,0.86,90.0,94.0,60.0,...,1.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,17.9
70206,A07,201407.0,NIH1604999732,75.0,1,1.06,1.02,134.0,128.0,66.0,...,1.0,1.0,NaN,NaN,18.81,NaN,NaN,NaN,NaN,10.5
70207,A07,201409.0,NIH1604999772,55.0,1,1.54,1.45,NaN,150.0,NaN,...,1.0,1.0,NaN,NaN,8.68,NaN,NaN,NaN,NaN,13.9
70208,A07,201403.0,NIH1604999929,53.0,2,0.89,0.87,110.0,114.0,70.0,...,1.0,1.0,NaN,NaN,3.47,NaN,NaN,NaN,NaN,27.1


In [10]:
df = calculate_physical_activity(df, 'PHYACTM')
df

<ipython-input-8-cc33c9591cc8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 210 / 60
<ipython-input-8-cc33c9591cc8>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 330 / 60
<ipython-input-8-cc33c9591cc8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 0
<ipython-input-8-cc33c9591cc8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,SBP_L,SBP_R,DBP_L,...,TREATD14,KID,KIDAG,KIDCU,TOTALC,MET,PHYACTL,PHYACTM,PHYACTH,BODYFAT
0,A01,200209.0,NIH1604000095,43.0,2,0.90,0.90,98.0,94.0,66.0,...,1.0,1.0,NaN,NaN,1.30,19740.0,4.50,3.50,4.0,13.4
1,A01,200201.0,NIH1604000171,42.0,1,1.00,1.00,110.0,80.0,80.0,...,1.0,1.0,NaN,NaN,42.68,17587.5,5.50,5.50,0.0,15.7
2,A01,200210.0,NIH1604000338,69.0,1,0.90,0.90,138.0,138.0,70.0,...,NaN,1.0,NaN,NaN,0.00,16380.0,1.25,0.00,7.0,NaN
3,A01,200201.0,NIH1604000362,48.0,2,0.70,0.60,104.0,100.0,76.0,...,NaN,1.0,NaN,NaN,0.00,7560.0,1.75,0.75,2.0,14.7
4,A01,200205.0,NIH1604000424,47.0,1,0.70,0.60,132.0,130.0,84.0,...,NaN,1.0,NaN,NaN,21.22,18375.0,5.50,5.50,0.0,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,0.88,0.86,90.0,94.0,60.0,...,1.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,17.9
70206,A07,201407.0,NIH1604999732,75.0,1,1.06,1.02,134.0,128.0,66.0,...,1.0,1.0,NaN,NaN,18.81,NaN,NaN,NaN,NaN,10.5
70207,A07,201409.0,NIH1604999772,55.0,1,1.54,1.45,NaN,150.0,NaN,...,1.0,1.0,NaN,NaN,8.68,NaN,NaN,NaN,NaN,13.9
70208,A07,201403.0,NIH1604999929,53.0,2,0.89,0.87,110.0,114.0,70.0,...,1.0,1.0,NaN,NaN,3.47,NaN,NaN,NaN,NaN,27.1


In [11]:
df = calculate_physical_activity(df, 'PHYACTH')
df

<ipython-input-8-cc33c9591cc8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 105 / 60
<ipython-input-8-cc33c9591cc8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 0
<ipython-input-8-cc33c9591cc8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname][idx] = 270 / 60
<ipython-input-8-cc33c9591cc8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,SBP_L,SBP_R,DBP_L,...,TREATD14,KID,KIDAG,KIDCU,TOTALC,MET,PHYACTL,PHYACTM,PHYACTH,BODYFAT
0,A01,200209.0,NIH1604000095,43.0,2,0.90,0.90,98.0,94.0,66.0,...,1.0,1.0,NaN,NaN,1.30,19740.0,4.50,3.50,1.75,13.4
1,A01,200201.0,NIH1604000171,42.0,1,1.00,1.00,110.0,80.0,80.0,...,1.0,1.0,NaN,NaN,42.68,17587.5,5.50,5.50,0.00,15.7
2,A01,200210.0,NIH1604000338,69.0,1,0.90,0.90,138.0,138.0,70.0,...,NaN,1.0,NaN,NaN,0.00,16380.0,1.25,0.00,4.50,NaN
3,A01,200201.0,NIH1604000362,48.0,2,0.70,0.60,104.0,100.0,76.0,...,NaN,1.0,NaN,NaN,0.00,7560.0,1.75,0.75,0.75,14.7
4,A01,200205.0,NIH1604000424,47.0,1,0.70,0.60,132.0,130.0,84.0,...,NaN,1.0,NaN,NaN,21.22,18375.0,5.50,5.50,0.00,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,0.88,0.86,90.0,94.0,60.0,...,1.0,1.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,17.9
70206,A07,201407.0,NIH1604999732,75.0,1,1.06,1.02,134.0,128.0,66.0,...,1.0,1.0,NaN,NaN,18.81,NaN,NaN,NaN,NaN,10.5
70207,A07,201409.0,NIH1604999772,55.0,1,1.54,1.45,NaN,150.0,NaN,...,1.0,1.0,NaN,NaN,8.68,NaN,NaN,NaN,NaN,13.9
70208,A07,201403.0,NIH1604999929,53.0,2,0.89,0.87,110.0,114.0,70.0,...,1.0,1.0,NaN,NaN,3.47,NaN,NaN,NaN,NaN,27.1


In [19]:
df['MET_CAL'] = df['PHYACTH']*8*60 + df['PHYACTM']*4*60 + df['PHYACTL']*3.3*60

In [20]:
df['MET_CAL'].describe()

count    9645.000000
mean     1316.847434
std      1088.080604
min         0.000000
25%       427.500000
50%      1089.000000
75%      2026.500000
max      5049.000000
Name: MET_CAL, dtype: float64

In [21]:
df['PA_NEW'] = 0
for idx in range(len(df)): 
    if df['MET_CAL'][idx] < 600: 
        df['PA_NEW'][idx] = 1
    elif df['MET_CAL'][idx] >= 600 and df['MET_CAL'][idx] <= 3000: 
        df['PA_NEW'][idx] = 2
    elif df['MET_CAL'][idx] > 3000: 
        df['PA_NEW'][idx] = 3
    print(df['MET_CAL'][idx], 'PA_NEW :', df['PA_NEW'][idx])

<ipython-input-21-63235a8bde0f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PA_NEW'][idx] = 2
<ipython-input-21-63235a8bde0f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PA_NEW'][idx] = 1
<ipython-input-21-63235a8bde0f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PA_NEW'][idx] = 3


2571.0 PA_NEW : 2
2409.0 PA_NEW : 2
2407.5 PA_NEW : 2
886.5 PA_NEW : 2
2409.0 PA_NEW : 2
2788.5 PA_NEW : 2
547.5 PA_NEW : 1
1567.5 PA_NEW : 2
495.0 PA_NEW : 1
60.0 PA_NEW : 1
3135.0 PA_NEW : 3
2655.0 PA_NEW : 2
667.5 PA_NEW : 2
148.49999999999997 PA_NEW : 1
2026.5 PA_NEW : 2
1089.0 PA_NEW : 2
247.5 PA_NEW : 1
2887.5 PA_NEW : 2
2013.0 PA_NEW : 2
1089.0 PA_NEW : 2
3333.0 PA_NEW : 3
2626.5 PA_NEW : 2
1089.0 PA_NEW : 2
891.0 PA_NEW : 2
1149.0 PA_NEW : 2
915.0 PA_NEW : 2
2788.5 PA_NEW : 2
nan PA_NEW : 0
2640.0 PA_NEW : 2
1089.0 PA_NEW : 2
1080.0 PA_NEW : 2
3046.5 PA_NEW : 3
1009.5 PA_NEW : 2
692.9999999999999 PA_NEW : 2
1089.0 PA_NEW : 2
3753.0 PA_NEW : 3
1828.5 PA_NEW : 2
946.5 PA_NEW : 2
1089.0 PA_NEW : 2
2848.5 PA_NEW : 2
1269.0 PA_NEW : 2
2211.0 PA_NEW : 2
495.0 PA_NEW : 1
3729.0 PA_NEW : 3
2409.0 PA_NEW : 2
1200.0 PA_NEW : 2
1275.0 PA_NEW : 2
1089.0 PA_NEW : 2
2689.5 PA_NEW : 2
1089.0 PA_NEW : 2
675.0 PA_NEW : 2
1147.5 PA_NEW : 2
2869.5 PA_NEW : 2
3046.5 PA_NEW : 3
706.5 PA_NEW : 2
136

49.5 PA_NEW : 1
526.5 PA_NEW : 1
646.5 PA_NEW : 2
667.5 PA_NEW : 2
495.0 PA_NEW : 1
247.5 PA_NEW : 1
692.9999999999999 PA_NEW : 2
4306.5 PA_NEW : 3
3729.0 PA_NEW : 3
1089.0 PA_NEW : 2
3046.5 PA_NEW : 3
3729.0 PA_NEW : 3
3333.0 PA_NEW : 3
3868.5 PA_NEW : 3
nan PA_NEW : 0
3453.0 PA_NEW : 3
1089.0 PA_NEW : 2
808.5 PA_NEW : 2
1629.0 PA_NEW : 2
1546.5 PA_NEW : 2
1689.0 PA_NEW : 2
2835.0 PA_NEW : 2
307.5 PA_NEW : 1
1507.5 PA_NEW : 2
247.5 PA_NEW : 1
148.49999999999997 PA_NEW : 1
3729.0 PA_NEW : 3
1089.0 PA_NEW : 2
nan PA_NEW : 0
766.5 PA_NEW : 2
346.49999999999994 PA_NEW : 1
891.0 PA_NEW : 2
49.5 PA_NEW : 1
1095.0 PA_NEW : 2
148.49999999999997 PA_NEW : 1
2013.0 PA_NEW : 2
2689.5 PA_NEW : 2
1893.0 PA_NEW : 2
692.9999999999999 PA_NEW : 2
692.9999999999999 PA_NEW : 2
49.5 PA_NEW : 1
307.5 PA_NEW : 1
406.49999999999994 PA_NEW : 1
1087.5 PA_NEW : 2
2368.5 PA_NEW : 2
2887.5 PA_NEW : 2
229.5 PA_NEW : 1
208.49999999999997 PA_NEW : 1
0.0 PA_NEW : 1
891.0 PA_NEW : 2
3007.5 PA_NEW : 3
1893.0 PA_NEW : 2

2026.5 PA_NEW : 2
1149.0 PA_NEW : 2
1468.5 PA_NEW : 2
208.49999999999997 PA_NEW : 1
675.0 PA_NEW : 2
247.5 PA_NEW : 1
307.5 PA_NEW : 1
2986.5 PA_NEW : 2
547.5 PA_NEW : 1
2788.5 PA_NEW : 2
568.5 PA_NEW : 1
495.0 PA_NEW : 1
109.5 PA_NEW : 1
49.5 PA_NEW : 1
1389.0 PA_NEW : 2
346.49999999999994 PA_NEW : 1
2151.0 PA_NEW : 2
247.5 PA_NEW : 1
120.0 PA_NEW : 1
526.5 PA_NEW : 1
891.0 PA_NEW : 2
787.5 PA_NEW : 2
667.5 PA_NEW : 2
2686.5 PA_NEW : 2
1389.0 PA_NEW : 2
1533.0 PA_NEW : 2
1929.0 PA_NEW : 2
2640.0 PA_NEW : 2
692.9999999999999 PA_NEW : 2
2689.5 PA_NEW : 2
795.0 PA_NEW : 2
0.0 PA_NEW : 1
2947.5 PA_NEW : 2
247.5 PA_NEW : 1
1549.5 PA_NEW : 2
3135.0 PA_NEW : 3
1335.0 PA_NEW : 2
3513.0 PA_NEW : 3
2887.5 PA_NEW : 2
1389.0 PA_NEW : 2
2409.0 PA_NEW : 2
208.49999999999997 PA_NEW : 1
1149.0 PA_NEW : 2
148.49999999999997 PA_NEW : 1
3393.0 PA_NEW : 3
1168.5 PA_NEW : 2
148.49999999999997 PA_NEW : 1
1567.5 PA_NEW : 2
49.5 PA_NEW : 1
247.5 PA_NEW : 1
1491.0 PA_NEW : 2
812.9999999999999 PA_NEW : 2
148.4

448.5 PA_NEW : 1
5049.0 PA_NEW : 3
1089.0 PA_NEW : 2
307.5 PA_NEW : 1
3729.0 PA_NEW : 3
148.49999999999997 PA_NEW : 1
247.5 PA_NEW : 1
1680.0 PA_NEW : 2
49.5 PA_NEW : 1
1689.0 PA_NEW : 2
346.49999999999994 PA_NEW : 1
1149.0 PA_NEW : 2
1395.0 PA_NEW : 2
1089.0 PA_NEW : 2
148.49999999999997 PA_NEW : 1
1228.5 PA_NEW : 2
2848.5 PA_NEW : 2
1987.5 PA_NEW : 2
1149.0 PA_NEW : 2
2887.5 PA_NEW : 2
307.5 PA_NEW : 1
5049.0 PA_NEW : 3
1447.5 PA_NEW : 2
891.0 PA_NEW : 2
3309.0 PA_NEW : 3
692.9999999999999 PA_NEW : 2
2308.5 PA_NEW : 2
346.49999999999994 PA_NEW : 1
1335.0 PA_NEW : 2
1089.0 PA_NEW : 2
1426.5 PA_NEW : 2
495.0 PA_NEW : 1
4108.5 PA_NEW : 3
148.49999999999997 PA_NEW : 1
208.49999999999997 PA_NEW : 1
466.49999999999994 PA_NEW : 1
555.0 PA_NEW : 1
2640.0 PA_NEW : 2
2788.5 PA_NEW : 2
nan PA_NEW : 0
951.0 PA_NEW : 2
0.0 PA_NEW : 1
1089.0 PA_NEW : 2
1246.5 PA_NEW : 2
2409.0 PA_NEW : 2
2788.5 PA_NEW : 2
247.5 PA_NEW : 1
1269.0 PA_NEW : 2
1149.0 PA_NEW : 2
692.9999999999999 PA_NEW : 2
148.4999999

...

nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW

nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW

nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW

nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW

nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW : 0
nan PA_NEW

In [22]:
df['PA_NEW'].describe()

count    70210.000000
mean         0.240550
std          0.642696
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          3.000000
Name: PA_NEW, dtype: float64

In [23]:
df

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,SBP_L,SBP_R,DBP_L,...,KIDAG,KIDCU,TOTALC,MET,PHYACTL,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW
0,A01,200209.0,NIH1604000095,43.0,2,0.90,0.90,98.0,94.0,66.0,...,NaN,NaN,1.30,19740.0,4.50,3.50,1.75,13.4,2571.0,2
1,A01,200201.0,NIH1604000171,42.0,1,1.00,1.00,110.0,80.0,80.0,...,NaN,NaN,42.68,17587.5,5.50,5.50,0.00,15.7,2409.0,2
2,A01,200210.0,NIH1604000338,69.0,1,0.90,0.90,138.0,138.0,70.0,...,NaN,NaN,0.00,16380.0,1.25,0.00,4.50,NaN,2407.5,2
3,A01,200201.0,NIH1604000362,48.0,2,0.70,0.60,104.0,100.0,76.0,...,NaN,NaN,0.00,7560.0,1.75,0.75,0.75,14.7,886.5,2
4,A01,200205.0,NIH1604000424,47.0,1,0.70,0.60,132.0,130.0,84.0,...,NaN,NaN,21.22,18375.0,5.50,5.50,0.00,19.5,2409.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,0.88,0.86,90.0,94.0,60.0,...,NaN,NaN,0.00,NaN,NaN,NaN,NaN,17.9,NaN,0
70206,A07,201407.0,NIH1604999732,75.0,1,1.06,1.02,134.0,128.0,66.0,...,NaN,NaN,18.81,NaN,NaN,NaN,NaN,10.5,NaN,0
70207,A07,201409.0,NIH1604999772,55.0,1,1.54,1.45,NaN,150.0,NaN,...,NaN,NaN,8.68,NaN,NaN,NaN,NaN,13.9,NaN,0
70208,A07,201403.0,NIH1604999929,53.0,2,0.89,0.87,110.0,114.0,70.0,...,NaN,NaN,3.47,NaN,NaN,NaN,NaN,27.1,NaN,0


In [24]:
# SBP, DBP 평균값으로 대체 후 L, R열 drop

df['SBP'] = (df['SBP_L'] + df['SBP_R']) / 2.0
df.drop(['SBP_L', 'SBP_R'], axis = 1, inplace = True)

In [25]:
df['DBP'] = (df['DBP_L'] + df['DBP_R']) / 2.0
df.drop(['DBP_L', 'DBP_R'], axis = 1, inplace = True)

In [26]:
# 크레아틴 기수별 전처리

df['CREA_ADJUSTED'] = 0.0

for idx in range(len(df)): 
    if df['기수'][idx] == 'A01' or df['기수'][idx] == 'A06' or df['기수'][idx] == 'A07': 
        df['CREA_ADJUSTED'][idx] = df['CREATININ_TR'][idx]
    else: 
        df['CREA_ADJUSTED'][idx] = df['CREATININ_ORI'][idx] 

<ipython-input-26-0db9417111ca>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CREA_ADJUSTED'][idx] = df['CREATININ_TR'][idx]
<ipython-input-26-0db9417111ca>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CREA_ADJUSTED'][idx] = df['CREATININ_ORI'][idx]


In [27]:
# eGFR 계산해서 추가
df['eGFR'] = 0.0

In [28]:
# eGFR 계산 함수

def calculate_eGFR(df): 
    tmpdf = df.copy()
    for i in range(len(tmpdf)): 
        if tmpdf['SEX'][i] == 1: # if male
            if tmpdf['CREA_ADJUSTED'][i] <= 0.9: 
                tmpdf['eGFR'][i] = 141*((tmpdf['CREA_ADJUSTED'][i]/0.9)**(-0.411))*(0.993**tmpdf['AGE'][i])
            else: 
                tmpdf['eGFR'][i] = 141*((tmpdf['CREA_ADJUSTED'][i]/0.9)**(-1.209))*(0.993**tmpdf['AGE'][i])
        if tmpdf['SEX'][i] == 2: # if female[i]
            if tmpdf['CREA_ADJUSTED'][i] <= 0.7: 
                tmpdf['eGFR'][i] = 144*((tmpdf['CREA_ADJUSTED'][i]/0.7)**(-0.329))*(0.993**tmpdf['AGE'][i])
            else: 
                tmpdf['eGFR'][i] = 144*((tmpdf['CREA_ADJUSTED'][i]/0.7)**(-1.209))*(0.993**tmpdf['AGE'][i])
                
    return tmpdf

In [29]:
df = calculate_eGFR(df)
df

<ipython-input-28-e6f1b093cf7a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmpdf['eGFR'][i] = 144*((tmpdf['CREA_ADJUSTED'][i]/0.7)**(-1.209))*(0.993**tmpdf['AGE'][i])
<ipython-input-28-e6f1b093cf7a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmpdf['eGFR'][i] = 141*((tmpdf['CREA_ADJUSTED'][i]/0.9)**(-1.209))*(0.993**tmpdf['AGE'][i])
<ipython-input-28-e6f1b093cf7a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmpdf['eGF

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,HEIGHT,WEIGHT,WAIST,...,PHYACTL,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,CREA_ADJUSTED,eGFR
0,A01,200209.0,NIH1604000095,43.0,2,0.90,0.90,157.5,48.0,63.0,...,4.50,3.50,1.75,13.4,2571.0,2,96.0,63.0,0.90,78.564168
1,A01,200201.0,NIH1604000171,42.0,1,1.00,1.00,175.5,75.0,81.3,...,5.50,5.50,0.00,15.7,2409.0,2,95.0,68.0,1.00,92.420166
2,A01,200210.0,NIH1604000338,69.0,1,0.90,0.90,166.8,50.4,74.0,...,1.25,0.00,4.50,NaN,2407.5,2,138.0,69.0,0.90,86.839489
3,A01,200201.0,NIH1604000362,48.0,2,0.70,0.60,155.6,53.6,76.4,...,1.75,0.75,0.75,14.7,886.5,2,102.0,74.0,0.70,102.784214
4,A01,200205.0,NIH1604000424,47.0,1,0.70,0.60,174.5,88.0,94.5,...,5.50,5.50,0.00,19.5,2409.0,2,131.0,84.0,0.70,112.380813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,0.88,0.86,164.1,60.9,74.0,...,NaN,NaN,NaN,17.9,NaN,0,92.0,62.0,0.88,74.201919
70206,A07,201407.0,NIH1604999732,75.0,1,1.06,1.02,163.2,50.8,81.3,...,NaN,NaN,NaN,10.5,NaN,0,131.0,66.0,1.06,68.312024
70207,A07,201409.0,NIH1604999772,55.0,1,1.54,1.45,163.5,66.2,88.3,...,NaN,NaN,NaN,13.9,NaN,0,NaN,NaN,1.54,50.048887
70208,A07,201403.0,NIH1604999929,53.0,2,0.89,0.87,153.0,63.2,85.3,...,NaN,NaN,NaN,27.1,NaN,0,112.0,71.0,0.89,74.230720


In [30]:
# BMI 계산

df['BMI'] = 0.0
df['HEIGHT'] = df['HEIGHT'] / 100

In [31]:
df['BMI'] = df['WEIGHT'] / df['HEIGHT']**2
df

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,HEIGHT,WEIGHT,WAIST,...,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,CREA_ADJUSTED,eGFR,BMI
0,A01,200209.0,NIH1604000095,43.0,2,0.90,0.90,1.575,48.0,63.0,...,3.50,1.75,13.4,2571.0,2,96.0,63.0,0.90,78.564168,19.349962
1,A01,200201.0,NIH1604000171,42.0,1,1.00,1.00,1.755,75.0,81.3,...,5.50,0.00,15.7,2409.0,2,95.0,68.0,1.00,92.420166,24.350452
2,A01,200210.0,NIH1604000338,69.0,1,0.90,0.90,1.668,50.4,74.0,...,0.00,4.50,NaN,2407.5,2,138.0,69.0,0.90,86.839489,18.115004
3,A01,200201.0,NIH1604000362,48.0,2,0.70,0.60,1.556,53.6,76.4,...,0.75,0.75,14.7,886.5,2,102.0,74.0,0.70,102.784214,22.138368
4,A01,200205.0,NIH1604000424,47.0,1,0.70,0.60,1.745,88.0,94.5,...,5.50,0.00,19.5,2409.0,2,131.0,84.0,0.70,112.380813,28.899599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,0.88,0.86,1.641,60.9,74.0,...,NaN,NaN,17.9,NaN,0,92.0,62.0,0.88,74.201919,22.615184
70206,A07,201407.0,NIH1604999732,75.0,1,1.06,1.02,1.632,50.8,81.3,...,NaN,NaN,10.5,NaN,0,131.0,66.0,1.06,68.312024,19.073193
70207,A07,201409.0,NIH1604999772,55.0,1,1.54,1.45,1.635,66.2,88.3,...,NaN,NaN,13.9,NaN,0,NaN,NaN,1.54,50.048887,24.764096
70208,A07,201403.0,NIH1604999929,53.0,2,0.89,0.87,1.530,63.2,85.3,...,NaN,NaN,27.1,NaN,0,112.0,71.0,0.89,74.230720,26.998163


In [32]:
df['TOTALC'].describe()

count    51142.000000
mean         9.108085
std         21.536243
min          0.000000
25%          0.000000
50%          0.000000
75%          7.520000
max        859.550000
Name: TOTALC, dtype: float64

In [37]:
# DRK_NEW 계산 함수

def calculate_DRK(df): 
    df['DRK_NEW'] = 0
    
    for idx in range(len(df)): 
    
        if df['TOTALC'][idx] == 0: 
            df['DRK_NEW'][idx] = 1
        
        elif df['TOTALC'][idx] < 10: 
            df['DRK_NEW'][idx] = 2
    
        elif df['TOTALC'][idx] < 20: 
            df['DRK_NEW'][idx] = 3
        
        elif df['TOTALC'][idx] < 40: 
            df['DRK_NEW'][idx] = 4
    
        elif df['TOTALC'][idx] >= 40: 
            df['DRK_NEW'][idx] = 5
        
        print(df['TOTALC'][idx], 'DRK_NEW: ', df['DRK_NEW'][idx])
            
    return df

In [38]:
df = calculate_DRK(df)
df

<ipython-input-37-1e6d459980a0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DRK_NEW'][idx] = 2
<ipython-input-37-1e6d459980a0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DRK_NEW'][idx] = 5
<ipython-input-37-1e6d459980a0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DRK_NEW'][idx] = 1
<ipython-input-37-1e6d459980a0>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

1.3 DRK_NEW:  2
42.68 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
21.22 DRK_NEW:  4
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.58 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
20.26 DRK_NEW:  4
0.0 DRK_NEW:  1
8.1 DRK_NEW:  2
4.48 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
2.69 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
63.96 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
1.16 DRK_NEW:  2
0.26 DRK_NEW:  2
20.26 DRK_NEW:  4
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
61.93 DRK_NEW:  5
11.0 DRK_NEW:  3
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
1.36 DRK_NEW:  2
8.68 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
8.88 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
6.95 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
21.42 DRK_NEW:  4
0.0 DRK_NEW:  1
7.57 DRK_NEW:  2
0.0 DRK_NEW:  1
4.78 DRK_NEW:  2
20.26 DRK_NEW:  4
63.67 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
49.89 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
14.47 DRK_NEW:  3
2.32 DRK_NEW:  2
0.0 DRK_NEW:  1
0.52 

0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
24.17 DRK_NEW:  4
11.58 DRK_NEW:  3
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
3.47 DRK_NEW:  2
0.0 DRK_NEW:  1
8.1 DRK_NEW:  2
0.81 DRK_NEW:  2
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
52.45 DRK_NEW:  5
1.65 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
3.43 DRK_NEW:  2
0.0 DRK_NEW:  1
nan DRK_NEW:  0
2.17 DRK_NEW:  2
40.52 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
2.55 DRK_NEW:  2
0.0 DRK_NEW:  1
22.99 DRK_NEW:  4
113.86 DRK_NEW:  5
1.3 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
5.04 DRK_NEW:  2
60.78 DRK_NEW:  5
0.0 DRK_NEW:  1
2.23 DRK_NEW:  2
92.61 DRK_NEW:  5
14.73 DRK_NEW:  3
0.0 DRK_NEW:  1
0.29 DRK_NEW:  2
0.0 DRK_NEW:  1
4.63 DRK_NEW:  2
8.68 DRK_NEW:  2
43.31 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
20.26 DRK_NEW:  4
0.0 DRK_NEW:  1
24.43 DRK_NEW:  4
0.0 DRK_NEW:  1
20.26 DRK_NEW:  4
0.0 DRK_NEW:  1
nan DRK_NEW:  0
28.94 DRK_NEW:  4
18.52 DRK_NEW:  3
1.65 DRK_NEW: 

0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
31.16 DRK_NEW:  4
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
17.36 DRK_NEW:  3
5.79 DRK_NEW:  2
5.43 DRK_NEW:  2
46.31 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
5.06 DRK_NEW:  2
0.0 DRK_NEW:  1
5.5 DRK_NEW:  2
0.0 DRK_NEW:  1
8.68 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
1.45 DRK_NEW:  2
10.19 DRK_NEW:  3
0.0 DRK_NEW:  1
17.36 DRK_NEW:  3
0.0 DRK_NEW:  1
3.47 DRK_NEW:  2
0.0 DRK_NEW:  1
63.96 DRK_NEW:  5
0.0 DRK_NEW:  1
13.76 DRK_NEW:  3
20.26 DRK_NEW:  4
2.1 DRK_NEW:  2
28.94 DRK_NEW:  4
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
1.16 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
15.63 DRK_NEW:  3
0.0 DRK_NEW:  1
20.26 DRK_NEW:  4
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
8.78 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.72 DRK_NEW:  2
1.04 DRK_

8.68 DRK_NEW:  2
0.0 DRK_NEW:  1
104.71 DRK_NEW:  5
11.58 DRK_NEW:  3
0.0 DRK_NEW:  1
48.49 DRK_NEW:  5
28.94 DRK_NEW:  4
0.85 DRK_NEW:  2
0.29 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
17.36 DRK_NEW:  3
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
60.78 DRK_NEW:  5
4.63 DRK_NEW:  2
0.0 DRK_NEW:  1
1.45 DRK_NEW:  2
11.58 DRK_NEW:  3
0.0 DRK_NEW:  1
8.1 DRK_NEW:  2
10.93 DRK_NEW:  3
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
30.97 DRK_NEW:  4
21.56 DRK_NEW:  4
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
4.63 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
10.61 DRK_NEW:  3
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
2.32 DRK_NEW:  2
14.47 DRK_NEW:  3
7.38 DRK_NEW:  2
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
10.42 DRK_NEW:  3
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
20.26 DRK_NEW:  4
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.52 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
26.87 DRK_NEW:  4
0.0 DRK_NEW:  1


...

0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
75.97 DRK_NEW:  5
nan DRK_NEW:  0
nan DRK_NEW:  0
28.94 DRK_NEW:  4
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
2.05 DRK_NEW:  2
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
71.53 DRK_NEW:  5
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
34.76 DRK_NEW:  4
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
11.37 DRK_NEW:  3
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
20.26 DRK_NEW:  4
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
1.16 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
1.5 DRK_NEW:  2
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NE

0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
26.73 DRK_NEW:  4
0.0 DRK_NEW:  1
nan DRK_NEW:  0
90.66 DRK_NEW:  5
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
6.99 DRK_NEW:  2
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
11.05 DRK_NEW:  3
15.05 DRK_NEW:  3
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
1.63 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
4.85 DRK_NEW:  2
nan DRK_NEW:  0
nan DRK_NEW:  0
3.47 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
2.17 DRK_NEW:  2
nan DRK_NEW:  0
0.0 DRK_NEW:  1
148.81 DRK_NEW:  5
nan DRK_NEW:  0
2.89 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
18.81 DRK_NEW:  3
nan DRK_NEW:  0
nan D

0.0 DRK_NEW:  1
34.73 DRK_NEW:  4
57.16 DRK_NEW:  5
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
57.88 DRK_NEW:  5
7.52 DRK_NEW:  2
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.67 DRK_NEW:  2
13.89 DRK_NEW:  3
nan DRK_NEW:  0
227.32 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.58 DRK_NEW:  2
nan DRK_NEW:  0
nan DRK_NEW:  0
6.06 DRK_NEW:  2
1.16 DRK_NEW:  2
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
1.3 DRK_NEW:  2
37.62 DRK_NEW:  4
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
1.45 DRK_NEW:  2
6.33 DRK_NEW:  2
52.09 DRK_NEW:  5
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
1.95 DRK_NEW:  2
0.58 DRK_NEW:  2
0.0 DRK_NEW:  1
nan DRK_NEW:  0
63.56 DRK_NEW:  5
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  

0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.65 DRK_NEW:  2
0.0 DRK_NEW:  1
nan DRK_NEW:  0
56.43 DRK_NEW:  5
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
2.32 DRK_NEW:  2
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
1.3 DRK_NEW:  2
0.38 DRK_NEW:  2
0.0 DRK_NEW:  1
0.87 DRK_NEW:  2
nan DRK_NEW:  0
34.73 DRK_NEW:  4
nan DRK_NEW:  0
4.56 DRK_NEW:  2
23.15 DRK_NEW:  4
nan DRK_NEW:  0
3.47 DRK_NEW:  2
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
nan DRK_NEW:  0
0.62 DRK_NEW:  2
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
18.81 DRK_NEW:  3
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_NEW:  1
0.0 DRK_NEW:  1
nan DRK_NEW:  0
0.0 DRK_NEW:  1
nan DRK_NEW:  0
nan DRK_NEW:  0
0.0 DRK_N

,기수,EDATE,NIHID,AGE,SEX,CREATININ_TR,CREATININ_ORI,HEIGHT,WEIGHT,WAIST,...,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,CREA_ADJUSTED,eGFR,BMI,DRK_NEW
0,A01,200209.0,NIH1604000095,43.0,2,0.90,0.90,1.575,48.0,63.0,...,1.75,13.4,2571.0,2,96.0,63.0,0.90,78.564168,19.349962,2
1,A01,200201.0,NIH1604000171,42.0,1,1.00,1.00,1.755,75.0,81.3,...,0.00,15.7,2409.0,2,95.0,68.0,1.00,92.420166,24.350452,5
2,A01,200210.0,NIH1604000338,69.0,1,0.90,0.90,1.668,50.4,74.0,...,4.50,NaN,2407.5,2,138.0,69.0,0.90,86.839489,18.115004,1
3,A01,200201.0,NIH1604000362,48.0,2,0.70,0.60,1.556,53.6,76.4,...,0.75,14.7,886.5,2,102.0,74.0,0.70,102.784214,22.138368,1
4,A01,200205.0,NIH1604000424,47.0,1,0.70,0.60,1.745,88.0,94.5,...,0.00,19.5,2409.0,2,131.0,84.0,0.70,112.380813,28.899599,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,0.88,0.86,1.641,60.9,74.0,...,NaN,17.9,NaN,0,92.0,62.0,0.88,74.201919,22.615184,1
70206,A07,201407.0,NIH1604999732,75.0,1,1.06,1.02,1.632,50.8,81.3,...,NaN,10.5,NaN,0,131.0,66.0,1.06,68.312024,19.073193,3
70207,A07,201409.0,NIH1604999772,55.0,1,1.54,1.45,1.635,66.2,88.3,...,NaN,13.9,NaN,0,NaN,NaN,1.54,50.048887,24.764096,2
70208,A07,201403.0,NIH1604999929,53.0,2,0.89,0.87,1.530,63.2,85.3,...,NaN,27.1,NaN,0,112.0,71.0,0.89,74.230720,26.998163,2


In [39]:
df.columns

Index(['기수', 'EDATE', 'NIHID', 'AGE', 'SEX', 'CREATININ_TR', 'CREATININ_ORI',
       'HEIGHT', 'WEIGHT', 'WAIST', 'GLU0_ORI', 'R_GTP_TR', 'AST_ORI',
       'ALT_ORI', 'TCHL_ORI', 'HDL_ORI', 'TRIGLY_ORI', 'HB_ORI', 'SMOKE',
       'DRUGINS', 'DRUGHT', 'TREATD5', 'DRUGICD', 'DRUGLP', 'FMHTN', 'FMHEA',
       'FMDM', 'PRT16_U', 'TREATD14', 'KID', 'KIDAG', 'KIDCU', 'TOTALC', 'MET',
       'PHYACTL', 'PHYACTM', 'PHYACTH', 'BODYFAT', 'MET_CAL', 'PA_NEW', 'SBP',
       'DBP', 'CREA_ADJUSTED', 'eGFR', 'BMI', 'DRK_NEW'],
      dtype='object')

In [40]:
# drop 할 column drop

df.drop(['CREATININ_TR', 'CREATININ_ORI', 'MET', 'CREA_ADJUSTED'], axis = 1, inplace = True)

In [41]:
df

,기수,EDATE,NIHID,AGE,SEX,HEIGHT,WEIGHT,WAIST,GLU0_ORI,R_GTP_TR,...,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,eGFR,BMI,DRK_NEW
0,A01,200209.0,NIH1604000095,43.0,2,1.575,48.0,63.0,91.0,16.0,...,3.50,1.75,13.4,2571.0,2,96.0,63.0,78.564168,19.349962,2
1,A01,200201.0,NIH1604000171,42.0,1,1.755,75.0,81.3,88.0,27.0,...,5.50,0.00,15.7,2409.0,2,95.0,68.0,92.420166,24.350452,5
2,A01,200210.0,NIH1604000338,69.0,1,1.668,50.4,74.0,82.0,52.0,...,0.00,4.50,NaN,2407.5,2,138.0,69.0,86.839489,18.115004,1
3,A01,200201.0,NIH1604000362,48.0,2,1.556,53.6,76.4,110.0,38.0,...,0.75,0.75,14.7,886.5,2,102.0,74.0,102.784214,22.138368,1
4,A01,200205.0,NIH1604000424,47.0,1,1.745,88.0,94.5,81.0,5.0,...,5.50,0.00,19.5,2409.0,2,131.0,84.0,112.380813,28.899599,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,1.641,60.9,74.0,88.0,NaN,...,NaN,NaN,17.9,NaN,0,92.0,62.0,74.201919,22.615184,1
70206,A07,201407.0,NIH1604999732,75.0,1,1.632,50.8,81.3,110.0,NaN,...,NaN,NaN,10.5,NaN,0,131.0,66.0,68.312024,19.073193,3
70207,A07,201409.0,NIH1604999772,55.0,1,1.635,66.2,88.3,85.0,NaN,...,NaN,NaN,13.9,NaN,0,NaN,NaN,50.048887,24.764096,2
70208,A07,201403.0,NIH1604999929,53.0,2,1.530,63.2,85.3,95.0,NaN,...,NaN,NaN,27.1,NaN,0,112.0,71.0,74.230720,26.998163,2


--------------------------------------------

In [31]:
tmp = df.loc[(df['기수'] == 'A01') | (df['기수'] == 'A05')].reset_index(drop = True)
tmp

,기수,EDATE,NIHID,AGE,SEX,HEIGHT,WEIGHT,WAIST,GLU0_ORI,R_GTP_TR,...,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,eGFR,BMI,DRK_NEW
0,A01,200209.0,NIH1604000095,43.0,2,1.575,48.0,63.0,91.0,16.0,...,3.50,1.75,13.4,2571.0,2,96.0,63.0,78.564168,19.349962,2
1,A01,200201.0,NIH1604000171,42.0,1,1.755,75.0,81.3,88.0,27.0,...,5.50,0.00,15.7,2409.0,2,95.0,68.0,92.420166,24.350452,5
2,A01,200210.0,NIH1604000338,69.0,1,1.668,50.4,74.0,82.0,52.0,...,0.00,4.50,NaN,2407.5,2,138.0,69.0,86.839489,18.115004,1
3,A01,200201.0,NIH1604000362,48.0,2,1.556,53.6,76.4,110.0,38.0,...,0.75,0.75,14.7,886.5,2,102.0,74.0,102.784214,22.138368,1
4,A01,200205.0,NIH1604000424,47.0,1,1.745,88.0,94.5,81.0,5.0,...,5.50,0.00,19.5,2409.0,2,131.0,84.0,112.380813,28.899599,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20055,A05,200909.0,NIH1604999643,51.0,2,1.650,60.1,70.2,77.0,NaN,...,NaN,NaN,16.7,NaN,3,89.0,58.0,80.731689,22.075298,1
20056,A05,201003.0,NIH1604999732,71.0,1,1.639,57.6,90.0,119.0,NaN,...,NaN,NaN,14.0,NaN,3,127.0,69.0,63.003582,21.441965,5
20057,A05,NaN,NIH1604999772,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,5
20058,A05,NaN,NIH1604999929,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,5


In [32]:
tmp.isna().sum()

기수                0
EDATE          3365
NIHID             0
AGE            3365
SEX               0
HEIGHT         3365
WEIGHT         3373
WAIST          3381
GLU0_ORI       3665
R_GTP_TR      10032
AST_ORI        3368
ALT_ORI        3373
TCHL_ORI       3368
HDL_ORI        3368
TRIGLY_ORI     3371
HB_ORI         3368
SMOKE          3511
DRUGINS        3967
DRUGHT         3967
TREATD5        8861
DRUGICD        3967
DRUGLP         3967
FMHTN          3365
FMHEA          3365
FMDM           3365
PRT16_U        3412
TREATD14       8861
KID            3384
KIDAG         19773
KIDCU         20043
TOTALC         3711
PHYACTL       10252
PHYACTM       10327
PHYACTH       10273
BODYFAT        5594
MET_CAL       10415
PA_NEW            0
SBP            3415
DBP            3415
eGFR           3383
BMI            3373
DRK_NEW           0
dtype: int64

In [33]:
baseline = df.loc[df['기수'] == 'A05'].reset_index(drop = True)
baseline

,기수,EDATE,NIHID,AGE,SEX,HEIGHT,WEIGHT,WAIST,GLU0_ORI,R_GTP_TR,...,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,eGFR,BMI,DRK_NEW
0,A05,201007.0,NIH1604000095,51.0,2,1.574,51.0,65.1,100.0,NaN,...,NaN,NaN,16.4,NaN,3,99.0,66.0,56.406072,20.585467,1
1,A05,200911.0,NIH1604000171,50.0,1,1.755,79.8,85.9,92.0,NaN,...,NaN,NaN,19.3,NaN,3,126.0,100.0,96.636512,25.908881,5
2,A05,201008.0,NIH1604000338,77.0,1,1.665,53.3,80.7,94.0,NaN,...,NaN,NaN,9.5,NaN,3,106.0,60.0,65.853842,19.226434,1
3,A05,200911.0,NIH1604000362,56.0,2,1.555,51.3,77.3,NaN,NaN,...,NaN,NaN,11.9,NaN,3,99.0,70.0,NaN,21.215662,1
4,A05,NaN,NIH1604000424,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10025,A05,200909.0,NIH1604999643,51.0,2,1.650,60.1,70.2,77.0,NaN,...,NaN,NaN,16.7,NaN,3,89.0,58.0,80.731689,22.075298,1
10026,A05,201003.0,NIH1604999732,71.0,1,1.639,57.6,90.0,119.0,NaN,...,NaN,NaN,14.0,NaN,3,127.0,69.0,63.003582,21.441965,5
10027,A05,NaN,NIH1604999772,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,5
10028,A05,NaN,NIH1604999929,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,5


In [34]:
baseline.isna().sum()

기수                0
EDATE          3365
NIHID             0
AGE            3365
SEX               0
HEIGHT         3365
WEIGHT         3368
WAIST          3373
GLU0_ORI       3366
R_GTP_TR      10030
AST_ORI        3366
ALT_ORI        3368
TCHL_ORI       3366
HDL_ORI        3366
TRIGLY_ORI     3368
HB_ORI         3366
SMOKE          3377
DRUGINS        3382
DRUGHT         3382
TREATD5        3379
DRUGICD        3382
DRUGLP         3382
FMHTN          3365
FMHEA          3365
FMDM           3365
PRT16_U        3375
TREATD14       3379
KID            3380
KIDAG         10014
KIDCU         10013
TOTALC         3405
PHYACTL       10030
PHYACTM       10030
PHYACTH       10030
BODYFAT        3403
MET_CAL       10030
PA_NEW            0
SBP            3396
DBP            3396
eGFR           3381
BMI            3368
DRK_NEW           0
dtype: int64

In [42]:
df.to_csv('MME_.csv', encoding = 'euc-kr', index = False)

-----------------------------------------

#### 1019_서원진T 드릴 데이터 다시 생성 (1기만 추출) 

In [43]:
df

,기수,EDATE,NIHID,AGE,SEX,HEIGHT,WEIGHT,WAIST,GLU0_ORI,R_GTP_TR,...,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,eGFR,BMI,DRK_NEW
0,A01,200209.0,NIH1604000095,43.0,2,1.575,48.0,63.0,91.0,16.0,...,3.50,1.75,13.4,2571.0,2,96.0,63.0,78.564168,19.349962,2
1,A01,200201.0,NIH1604000171,42.0,1,1.755,75.0,81.3,88.0,27.0,...,5.50,0.00,15.7,2409.0,2,95.0,68.0,92.420166,24.350452,5
2,A01,200210.0,NIH1604000338,69.0,1,1.668,50.4,74.0,82.0,52.0,...,0.00,4.50,NaN,2407.5,2,138.0,69.0,86.839489,18.115004,1
3,A01,200201.0,NIH1604000362,48.0,2,1.556,53.6,76.4,110.0,38.0,...,0.75,0.75,14.7,886.5,2,102.0,74.0,102.784214,22.138368,1
4,A01,200205.0,NIH1604000424,47.0,1,1.745,88.0,94.5,81.0,5.0,...,5.50,0.00,19.5,2409.0,2,131.0,84.0,112.380813,28.899599,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,201303.0,NIH1604999643,55.0,2,1.641,60.9,74.0,88.0,NaN,...,NaN,NaN,17.9,NaN,0,92.0,62.0,74.201919,22.615184,1
70206,A07,201407.0,NIH1604999732,75.0,1,1.632,50.8,81.3,110.0,NaN,...,NaN,NaN,10.5,NaN,0,131.0,66.0,68.312024,19.073193,3
70207,A07,201409.0,NIH1604999772,55.0,1,1.635,66.2,88.3,85.0,NaN,...,NaN,NaN,13.9,NaN,0,NaN,NaN,50.048887,24.764096,2
70208,A07,201403.0,NIH1604999929,53.0,2,1.530,63.2,85.3,95.0,NaN,...,NaN,NaN,27.1,NaN,0,112.0,71.0,74.230720,26.998163,2


In [44]:
dfA01 = df.loc[df['기수'] == 'A01']
dfA01

,기수,EDATE,NIHID,AGE,SEX,HEIGHT,WEIGHT,WAIST,GLU0_ORI,R_GTP_TR,...,PHYACTM,PHYACTH,BODYFAT,MET_CAL,PA_NEW,SBP,DBP,eGFR,BMI,DRK_NEW
0,A01,200209.0,NIH1604000095,43.0,2,1.575,48.0,63.0,91.0,16.0,...,3.50,1.75,13.4,2571.0,2,96.0,63.0,78.564168,19.349962,2
1,A01,200201.0,NIH1604000171,42.0,1,1.755,75.0,81.3,88.0,27.0,...,5.50,0.00,15.7,2409.0,2,95.0,68.0,92.420166,24.350452,5
2,A01,200210.0,NIH1604000338,69.0,1,1.668,50.4,74.0,82.0,52.0,...,0.00,4.50,NaN,2407.5,2,138.0,69.0,86.839489,18.115004,1
3,A01,200201.0,NIH1604000362,48.0,2,1.556,53.6,76.4,110.0,38.0,...,0.75,0.75,14.7,886.5,2,102.0,74.0,102.784214,22.138368,1
4,A01,200205.0,NIH1604000424,47.0,1,1.745,88.0,94.5,81.0,5.0,...,5.50,0.00,19.5,2409.0,2,131.0,84.0,112.380813,28.899599,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10025,A01,200109.0,NIH1604999643,43.0,2,1.644,61.0,71.3,85.0,8.0,...,2.50,0.00,16.2,1689.0,2,99.0,71.0,90.587425,22.569722,1
10026,A01,200203.0,NIH1604999732,63.0,1,1.650,56.0,82.5,133.0,436.0,...,0.00,5.50,NaN,2788.5,2,140.0,98.0,95.070429,20.569330,3
10027,A01,200301.0,NIH1604999772,44.0,1,1.640,68.5,83.0,91.0,115.0,...,0.00,0.00,17.2,495.0,1,128.0,94.0,81.212239,25.468471,5
10028,A01,200206.0,NIH1604999929,41.0,2,1.540,59.0,74.7,90.0,14.0,...,0.00,0.00,18.1,693.0,2,102.0,74.0,107.964675,24.877720,2


In [45]:
dfA01.to_csv('MME_A01_1019.csv', encoding = 'euc-kr', index = False)